In [1]:
import pandas as pd 
import datetime
data = pd.read_csv('data.csv')
user_id_list = data['user_id'].unique()
user_id_list

array([    0,     1,     2, ..., 26516, 26517, 26518])

In [2]:
new_data = {'user_id': user_id_list, 'car_ownership': 0}
df = pd.DataFrame(new_data)
df

,user_id,car_ownership
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
26514,26514,0
26515,26515,0
26516,26516,0
26517,26517,0


In [3]:
tripleg = pd.read_csv('triplegs.csv')
stop_points = pd.read_csv('staypoints.csv')

In [4]:
stop_points['started_at'] = pd.to_datetime(stop_points['started_at'])
tripleg['finished_at'] = pd.to_datetime(tripleg['finished_at'])

In [5]:
ID_list=stop_points['user_id'].unique()
len(ID_list)

11999

In [6]:
time_window = datetime.timedelta(minutes=5)

miss = 0
check_len = 0
data_to_append_list = []
for id in ID_list:
    related_stop_by_user = stop_points[stop_points['user_id'] == id]
    related_tripled_by_user = tripleg[tripleg['user_id'] == id]
    if len(related_tripled_by_user) == 0: 
        miss += 1
        continue
    
    for stop_related_row in related_stop_by_user.itertuples():
        tripled_found = related_tripled_by_user[
            (related_tripled_by_user['finished_at'] > stop_related_row[3] - time_window) &
            (related_tripled_by_user['finished_at'] <= stop_related_row[3])
        ]
        #print(len(tripled_found))
        if len(tripled_found) == 0: continue #did not found related trip
        #check if there if repeat data
        if len(tripled_found) > 1: 
            check_len += 1
            tripled_found = tripled_found.nlargest(1, 'finished_at')
        
        data_to_append = {'user_id': id, 'trip_started_at': tripled_found.iloc[0]['started_at'],
                'trip_finished_at': tripled_found.iloc[0]['finished_at'],
                'trip':tripled_found.iloc[0]['geom']}
        
        data_to_append_list.append(data_to_append)
        

trip_df = pd.DataFrame(data_to_append_list, columns=['user_id', 'trip_started_at', 'trip_finished_at', 'trip'])
trip_df

,user_id,trip_started_at,trip_finished_at,trip
0,1,2022-09-05 00:30:32-04:00,2022-09-05 00:31:43-04:00,LINESTRING (-81.6840667799999949 29.6517639099...
1,1,2022-09-05 00:36:45-04:00,2022-09-05 00:38:51-04:00,LINESTRING (-81.6578598100000050 29.6550407400...
2,1,2022-09-05 00:43:14-04:00,2022-09-05 00:53:20-04:00,LINESTRING (-81.6578598100000050 29.6550407400...
3,1,2022-09-05 01:57:58-04:00,2022-09-05 01:59:17-04:00,LINESTRING (-81.6578598100000050 29.6550407400...
4,1,2022-09-05 02:32:11-04:00,2022-09-05 02:32:58-04:00,LINESTRING (-81.6582779999999957 29.6534930000...
...,...,...,...,...
138514,26497,2022-10-30 02:44:10-04:00,2022-10-30 02:44:25-04:00,LINESTRING (-83.0702699999999936 30.5340899999...
138515,26498,2022-10-26 23:03:49-04:00,2022-10-26 23:04:01-04:00,LINESTRING (-81.5443425800000057 30.3223002299...
138516,26507,2022-10-25 01:31:14-04:00,2022-10-25 01:31:48-04:00,LINESTRING (-81.2006099999999975 29.4927099999...
138517,26507,2022-10-25 02:34:46-04:00,2022-10-25 02:37:00-04:00,LINESTRING (-81.2075099999999992 29.4838099999...


In [7]:
trip_df.to_csv('stop_related_trip.csv')

In [8]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas
import trackintel as ti
import numpy as np
tpls = ti.io.file.read_triplegs_csv('stop_related_trip.csv', columns={'trip_started_at':'started_at', 'trip_finished_at':'finished_at', 'trip':'geom'}, crs='EPSG:4326', index_col=0)

In [9]:
# Predict the transport mode of triplegs.
"""simple-coarse method includes {'slow_mobility', 'motorized_mobility', 'fast_mobility'}. 
In the default classification, slow_mobility (<15 km/h) includes transport modes such as walking or cycling, 
motorized_mobility (<100 km/h) modes such as car or train, 
and fast_mobility (>100 km/h) modes such as high-speed rail or airplanes."""
categories = {
    15/3.6: 'slow_mobility',  # walking or cycling
    193/3.6: 'motorized_mobility',  # car
    np.inf: 'fast_mobility'  # high-speed rail or airplane
}
mode_tpls  = tpls.as_triplegs.predict_transport_mode(categories = categories)
mode_tpls

/usr/local/lib/python3.9/site-packages/pygeos/io.py:89: UserWarning: The shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the PyGEOS GEOS version (3.10.4-CAPI-1.16.2). Conversions between both will be slow
  warnings.warn(


,user_id,started_at,finished_at,geom,mode
0,1,2022-09-05 00:30:32-04:00,2022-09-05 00:31:43-04:00,"LINESTRING (-81.68407 29.65176, -81.65786 29.6...",fast_mobility
1,1,2022-09-05 00:36:45-04:00,2022-09-05 00:38:51-04:00,"LINESTRING (-81.65786 29.65504, -81.68407 29.6...",motorized_mobility
2,1,2022-09-05 00:43:14-04:00,2022-09-05 00:53:20-04:00,"LINESTRING (-81.65786 29.65504, -81.65784 29.6...",fast_mobility
3,1,2022-09-05 01:57:58-04:00,2022-09-05 01:59:17-04:00,"LINESTRING (-81.65786 29.65504, -81.68407 29.6...",fast_mobility
4,1,2022-09-05 02:32:11-04:00,2022-09-05 02:32:58-04:00,"LINESTRING (-81.65828 29.65349, -81.67752 29.6...",motorized_mobility
...,...,...,...,...,...
138514,26497,2022-10-30 02:44:10-04:00,2022-10-30 02:44:25-04:00,"LINESTRING (-83.07027 30.53409, -83.07025 30.5...",slow_mobility
138515,26498,2022-10-26 23:03:49-04:00,2022-10-26 23:04:01-04:00,"LINESTRING (-81.54434 30.32230, -81.55156 30.3...",fast_mobility
138516,26507,2022-10-25 01:31:14-04:00,2022-10-25 01:31:48-04:00,"LINESTRING (-81.20061 29.49271, -81.21041 29.4...",motorized_mobility
138517,26507,2022-10-25 02:34:46-04:00,2022-10-25 02:37:00-04:00,"LINESTRING (-81.20751 29.48381, -81.21092 29.4...",motorized_mobility


In [10]:
mode_split = ti.analysis.modal_split.calculate_modal_split(tpls=mode_tpls, metric='count', per_user=True)
mode_split

mode,fast_mobility,motorized_mobility,slow_mobility
user_id,,,
1,7,2,1
4,5,0,4
5,0,1,3
8,0,1,0
10,0,2,0
...,...,...,...
26477,0,0,1
26482,0,1,0
26497,2,15,7


In [11]:
index = mode_split.index.values
for id in index:
    motor_count = mode_split.loc[id, 'fast_mobility'] + mode_split.loc[id, 'motorized_mobility'] + mode_split.loc[id, 'slow_mobility']
    if mode_split.loc[id, 'motorized_mobility'] / motor_count > 0.5:
        df.loc[df['user_id'] == id, 'car_ownership'] = 1
df

,user_id,car_ownership
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
26514,26514,0
26515,26515,0
26516,26516,0
26517,26517,0


In [12]:
print(len(df[df['car_ownership'] == 1]))

3214


In [13]:
home_location = pd.read_csv('home_location.csv', index_col=0)
home_location

,ID,LAT,LON,LAT-4326,LON-4326
0,0,3401080.0,-9141180.0,29.199637,-82.116617
1,1,3458860.0,-9093020.0,29.651722,-81.683988
2,2,3456360.0,-9163720.0,29.632203,-82.319097
3,3,3531380.0,-9073260.0,30.216272,-81.506481
4,4,3510020.0,-9105740.0,30.050323,-81.798254
...,...,...,...,...,...
22020,26492,3372900.0,-9048460.0,28.978423,-81.283699
22021,26501,3588800.0,-9065680.0,30.660990,-81.438389
22022,26503,3490020.0,-9051880.0,29.894687,-81.314422
22023,26506,3545180.0,-9097480.0,30.323338,-81.724053


In [14]:
from geopy.distance import geodesic
from shapely.wkt import loads
from tqdm import tqdm
data_to_append_list = []
home_id_list = home_location['ID'].unique()
for id in tqdm(home_id_list, total = len(home_id_list)):
    # print(home_location.loc[home_location['ID'] == id, 'LAT-4326'].values[0])
    p1 = (home_location.loc[home_location['ID'] == id, "LAT-4326"].values[0], home_location.loc[home_location['ID'] == id, "LON-4326"].values[0])
    # print(p1)
    for sp in stop_points[stop_points['user_id'] == id].itertuples():
        p = loads(sp[5])
        lat = p.y
        lon = p.x
        p2 = (lat, lon)
        # print(p2)
        if geodesic(p1, p2).meters < 100:
            data_to_append_list.append(sp)
new_staypoints = pd.DataFrame.from_records(data_to_append_list)
new_staypoints


100%|██████████| 22025/22025 [02:15<00:00, 162.10it/s]


,0,1,2,3,4,5
0,1,1,1,2022-09-05 00:31:55-04:00,2022-09-05 00:36:45-04:00,POINT (-81.6840837050000061 29.6517493699999974)
1,2,2,1,2022-09-05 00:38:57-04:00,2022-09-05 00:43:14-04:00,POINT (-81.6840733899999947 29.6517569549999997)
2,3,3,1,2022-09-05 00:53:25-04:00,2022-09-05 00:57:50-04:00,POINT (-81.6840733899999947 29.6517619549999978)
3,4,4,1,2022-09-05 01:47:48-04:00,2022-09-05 01:57:58-04:00,POINT (-81.6840667799999949 29.6517639099999997)
4,9,9,1,2022-09-06 01:49:19-04:00,2022-09-06 01:58:55-04:00,POINT (-81.6840667799999949 29.6517639099999997)
...,...,...,...,...,...,...
98059,586147,586147,25883,2022-10-30 18:13:02-04:00,2022-10-30 18:21:52-04:00,POINT (-81.6611999999999938 30.6507750000000030)
98060,586153,586153,25888,2022-10-22 13:59:34-04:00,2022-10-22 14:11:06-04:00,POINT (-82.3622717500000050 29.5906182000000015)
98061,586306,586306,25943,2022-10-30 13:52:38-04:00,2022-10-30 14:03:03-04:00,POINT (-81.4608493900000070 30.6265586600000006)
98062,586849,586849,26157,2022-10-24 12:32:31-04:00,2022-10-24 12:45:35-04:00,POINT (-81.6479450000000071 28.9665500000000016)


In [15]:
drop_cols = [0,1]
new_staypoint = new_staypoints.drop(new_staypoints.columns[drop_cols], axis=1)
new_staypoint = new_staypoint.rename(columns={2:'user_id', 3: 'started_at', 4: 'finished_at', 5: 'geom'})
new_staypoint


,user_id,started_at,finished_at,geom
0,1,2022-09-05 00:31:55-04:00,2022-09-05 00:36:45-04:00,POINT (-81.6840837050000061 29.6517493699999974)
1,1,2022-09-05 00:38:57-04:00,2022-09-05 00:43:14-04:00,POINT (-81.6840733899999947 29.6517569549999997)
2,1,2022-09-05 00:53:25-04:00,2022-09-05 00:57:50-04:00,POINT (-81.6840733899999947 29.6517619549999978)
3,1,2022-09-05 01:47:48-04:00,2022-09-05 01:57:58-04:00,POINT (-81.6840667799999949 29.6517639099999997)
4,1,2022-09-06 01:49:19-04:00,2022-09-06 01:58:55-04:00,POINT (-81.6840667799999949 29.6517639099999997)
...,...,...,...,...
98059,25883,2022-10-30 18:13:02-04:00,2022-10-30 18:21:52-04:00,POINT (-81.6611999999999938 30.6507750000000030)
98060,25888,2022-10-22 13:59:34-04:00,2022-10-22 14:11:06-04:00,POINT (-82.3622717500000050 29.5906182000000015)
98061,25943,2022-10-30 13:52:38-04:00,2022-10-30 14:03:03-04:00,POINT (-81.4608493900000070 30.6265586600000006)
98062,26157,2022-10-24 12:32:31-04:00,2022-10-24 12:45:35-04:00,POINT (-81.6479450000000071 28.9665500000000016)


In [16]:
time_window = datetime.timedelta(minutes=5)

miss = 0
check_len = 0
data_to_append_list = []
for id in ID_list:
    related_stop_by_user = new_staypoint[new_staypoint['user_id'] == id]
    related_tripled_by_user = tripleg[tripleg['user_id'] == id]
    if len(related_tripled_by_user) == 0: 
        miss += 1
        continue
    
    for stop_related_row in related_stop_by_user.itertuples():
        tripled_found = related_tripled_by_user[
            (related_tripled_by_user['finished_at'] > stop_related_row[2] - time_window) &
            (related_tripled_by_user['finished_at'] <= stop_related_row[2])
        ]
        #print(len(tripled_found))
        if len(tripled_found) == 0: continue #did not found related trip
        #check if there if repeat data
        if len(tripled_found) > 1: 
            check_len += 1
            tripled_found = tripled_found.nlargest(1, 'finished_at')
        
        data_to_append = {'user_id': id, 'trip_started_at': tripled_found.iloc[0]['started_at'],
                'trip_finished_at': tripled_found.iloc[0]['finished_at'],
                'trip':tripled_found.iloc[0]['geom']}
        
        data_to_append_list.append(data_to_append)
        

new_trip_df = pd.DataFrame(data_to_append_list, columns=['user_id', 'trip_started_at', 'trip_finished_at', 'trip'])
new_trip_df

,user_id,trip_started_at,trip_finished_at,trip
0,1,2022-09-05 00:30:32-04:00,2022-09-05 00:31:43-04:00,LINESTRING (-81.6840667799999949 29.6517639099...
1,1,2022-09-05 00:36:45-04:00,2022-09-05 00:38:51-04:00,LINESTRING (-81.6578598100000050 29.6550407400...
2,1,2022-09-05 00:43:14-04:00,2022-09-05 00:53:20-04:00,LINESTRING (-81.6578598100000050 29.6550407400...
3,1,2022-09-06 01:48:53-04:00,2022-09-06 01:48:59-04:00,LINESTRING (-81.6840667799999949 29.6517639099...
4,1,2022-09-06 01:58:55-04:00,2022-09-06 02:01:26-04:00,LINESTRING (-81.6578598100000050 29.6550407400...
...,...,...,...,...
15322,25907,2022-10-25 03:14:30-04:00,2022-10-25 03:14:52-04:00,LINESTRING (-81.7429900000000060 30.3809600000...
15323,26315,2022-10-27 18:08:05-04:00,2022-10-27 18:08:52-04:00,LINESTRING (-82.0434308100000038 29.7534871099...
15324,26320,2022-10-31 18:05:11-04:00,2022-10-31 18:07:22-04:00,LINESTRING (-82.1679170000000028 29.1673000000...
15325,26320,2022-10-31 22:05:24-04:00,2022-10-31 22:05:27-04:00,LINESTRING (-82.1617130000000060 29.1748960000...


In [17]:
new_trip_df.to_csv('home_related_trip.csv')

In [18]:
new_tpls = ti.io.file.read_triplegs_csv('home_related_trip.csv', columns={'trip_started_at':'started_at', 'trip_finished_at':'finished_at', 'trip':'geom'}, crs='EPSG:4326', index_col=0)
new_mode_tpls  = new_tpls.as_triplegs.predict_transport_mode(categories = categories)
new_mode_tpls

,user_id,started_at,finished_at,geom,mode
0,1,2022-09-05 00:30:32-04:00,2022-09-05 00:31:43-04:00,"LINESTRING (-81.68407 29.65176, -81.65786 29.6...",fast_mobility
1,1,2022-09-05 00:36:45-04:00,2022-09-05 00:38:51-04:00,"LINESTRING (-81.65786 29.65504, -81.68407 29.6...",motorized_mobility
2,1,2022-09-05 00:43:14-04:00,2022-09-05 00:53:20-04:00,"LINESTRING (-81.65786 29.65504, -81.65784 29.6...",fast_mobility
3,1,2022-09-06 01:48:53-04:00,2022-09-06 01:48:59-04:00,"LINESTRING (-81.68407 29.65176, -81.65786 29.6...",fast_mobility
4,1,2022-09-06 01:58:55-04:00,2022-09-06 02:01:26-04:00,"LINESTRING (-81.65786 29.65504, -81.68407 29.6...",fast_mobility
...,...,...,...,...,...
15322,25907,2022-10-25 03:14:30-04:00,2022-10-25 03:14:52-04:00,"LINESTRING (-81.74299 30.38096, -81.74298 30.3...",motorized_mobility
15323,26315,2022-10-27 18:08:05-04:00,2022-10-27 18:08:52-04:00,"LINESTRING (-82.04343 29.75349, -82.04197 29.7...",motorized_mobility
15324,26320,2022-10-31 18:05:11-04:00,2022-10-31 18:07:22-04:00,"LINESTRING (-82.16792 29.16730, -82.16792 29.1...",motorized_mobility
15325,26320,2022-10-31 22:05:24-04:00,2022-10-31 22:05:27-04:00,"LINESTRING (-82.16171 29.17490, -82.16170 29.1...",slow_mobility


In [19]:
new_mode_split = ti.analysis.modal_split.calculate_modal_split(tpls=new_mode_tpls, metric='count', per_user=True)
new_mode_split

mode,fast_mobility,motorized_mobility,slow_mobility
user_id,,,
1,4,1,0
4,0,0,1
14,7,17,10
33,0,0,1
51,2,16,5
...,...,...,...
25853,0,0,1
25907,0,2,1
26315,0,1,0


In [20]:
index = new_mode_split.index.values
for id in index:
    motor_count = mode_split.loc[id, 'motorized_mobility']
    if motor_count > 0:
        df.loc[df['user_id'] == id, 'car_ownership'] = 1
df

,user_id,car_ownership
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0
...,...,...
26514,26514,0
26515,26515,0
26516,26516,0
26517,26517,0


In [21]:
print(len(df[df['car_ownership'] == 1]))

3908


In [22]:
home_locations = home_location.rename(columns={'ID':'user_id'})
home_locations
merged_data = pd.merge(df, home_locations[['user_id','LAT-4326','LON-4326']], on='user_id', how='left')
merged_data

,user_id,car_ownership,LAT-4326,LON-4326
0,0,0,29.199637,-82.116617
1,1,1,29.651722,-81.683988
2,2,0,29.632203,-82.319097
3,3,0,30.216272,-81.506481
4,4,0,30.050323,-81.798254
...,...,...,...,...
26514,26514,0,29.988720,-81.480610
26515,26515,0,30.379152,-81.605476
26516,26516,0,30.336054,-81.757291
26517,26517,0,29.409107,-82.247951


In [23]:
merged_data.to_csv('User.csv')